In [6]:
import pandas as pd
#!conda install pytorch torchvision torchaudio -c pytorch --yes
# Load the data
file_path = '/home/yui/Downloads/QNN-test/pjm_east_2024_synthetic.csv'
data = pd.read_csv(file_path)

In [7]:
# Feature target split
target = data['value']
features = data.drop('value', axis=1)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset
# Good old train test split
# Convert features and target to tensors
X = torch.tensor(features.values, dtype=torch.float32)
y = torch.tensor(target.values, dtype=torch.float32).unsqueeze(1)  # Ensure y is the correct shape
# Initialize the scaler
scaler = StandardScaler()
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the scaler on your TRAINING data only
X_train = scaler.fit_transform(X_train.numpy())  # Convert to NumPy array to fit
X_test = scaler.transform(X_test.numpy())  # Apply the same transform to the test data

# Convert scaled features back to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

# Create DataLoader for training
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


val_dataset = TensorDataset(X_test, y_test)

In [9]:
import torch.nn as nn
import torch.optim as optim

class QuantileNetwork(nn.Module):
    def __init__(self, input_size, num_quantiles):
        super(QuantileNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_quantiles)
        )

    def forward(self, x):
        return self.network(x)

def quantile_loss(preds, target, quantiles):
    assert len(quantiles) == preds.shape[1], "Quantiles size must match predictions width."
    errors = target.unsqueeze(1) - preds
    return torch.max((quantiles - 1) * errors, quantiles * errors).mean()

# Initialize the model
num_features = X_train.shape[1]
num_quantiles = 50
quantiles = torch.linspace(0.01, 0.99, steps=num_quantiles)
model = QuantileNetwork(input_size=num_features, num_quantiles=num_quantiles)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        preds = model(batch_x)
        loss = quantile_loss(preds, batch_y, quantiles)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 6364814848.0
Epoch 2, Loss: 6415205888.0
Epoch 3, Loss: 6291462656.0
Epoch 4, Loss: 6314597888.0
Epoch 5, Loss: 6131255808.0
Epoch 6, Loss: 5665738752.0
Epoch 7, Loss: 6200896512.0
Epoch 8, Loss: 6032240640.0
Epoch 9, Loss: 6243320320.0
Epoch 10, Loss: 6124221440.0
Epoch 11, Loss: 5662082560.0
Epoch 12, Loss: 6196228096.0
Epoch 13, Loss: 6352038912.0
Epoch 14, Loss: 6006970880.0
Epoch 15, Loss: 6249884672.0
Epoch 16, Loss: 6360613376.0
Epoch 17, Loss: 6407648768.0
Epoch 18, Loss: 6347507712.0
Epoch 19, Loss: 6797496320.0
Epoch 20, Loss: 6206689792.0
Epoch 21, Loss: 6032711168.0
Epoch 22, Loss: 6304288768.0
Epoch 23, Loss: 5556603904.0
Epoch 24, Loss: 5939987968.0
Epoch 25, Loss: 6186130432.0
Epoch 26, Loss: 6089611264.0
Epoch 27, Loss: 6429102080.0
Epoch 28, Loss: 6134003200.0
Epoch 29, Loss: 6332724224.0
Epoch 30, Loss: 6097154048.0
Epoch 31, Loss: 6354560000.0
Epoch 32, Loss: 6327782400.0
Epoch 33, Loss: 6304242176.0
Epoch 34, Loss: 5821998080.0
Epoch 35, Loss: 6051617

In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
num_features = X_train.shape[1]
num_quantiles = 50
quantiles = torch.linspace(0.01, 0.99, steps=num_quantiles)
def quantile_loss(preds, target, quantiles):
    assert len(quantiles) == preds.shape[1], "Quantiles size must match predictions width."
    errors = target.unsqueeze(1) - preds
    return torch.max((quantiles - 1) * errors, quantiles * errors).mean()
# Define Quantile Network with three hidden layers
class QuantileNetwork(nn.Module):
    def __init__(self, input_size, output_size, hidden_sizes, dropout_rate):
        super(QuantileNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_sizes)):
            if i == 0:
                layers.append(nn.Linear(input_size, hidden_sizes[i]))
            else:
                layers.append(nn.Linear(hidden_sizes[i - 1], hidden_sizes[i]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))

        layers.append(nn.Linear(hidden_sizes[-1], output_size))  # Output layer
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Train and evaluate model
def train_and_evaluate(model, train_loader, val_loader, optimizer, quantiles, epochs=2000):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = quantile_loss(output, target, quantiles)
            loss.backward()
            optimizer.step()

    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            loss = quantile_loss(output, target, quantiles)
            total_loss += loss.item()
    avg_loss = total_loss / len(val_loader)
    return avg_loss

# Hyperparameter search space adjustments
neurons_options = [
    (32, 64, 128),
    (64, 128, 256),
    (128, 256, 512)
]
batch_sizes = [128, 512]#[64, 128, 256, 512]
neurons = [32, 64, 128, 256]
optimizers_dict = {'Adam': optim.Adam, 'Adagrad': optim.Adagrad, 'RMSprop': optim.RMSprop}
dropout_rates = np.linspace(0, 0.3, num=4)
learning_rates = np.logspace(-5, -1, num=5)
# Calculate the total number of iterations
total_iterations = len(batch_sizes) * len(neurons_options) * len(dropout_rates) * len(optimizers_dict) * len(learning_rates)
current_iteration = 0
# Hyperparameter search
best_loss = np.inf
best_config = None
best_model=None
for batch_size in batch_sizes:
    for hidden_sizes in neurons_options:  # Corrected from neurons to neurons_options
        for dropout_rate in dropout_rates:
            for optimizer_name, optimizer_class in optimizers_dict.items():
                for lr in learning_rates:
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
                    # Correct instantiation with the actual input size
                    model = QuantileNetwork(input_size=85, output_size=50, hidden_sizes=hidden_sizes, dropout_rate=dropout_rate)
                    optimizer = optimizer_class(model.parameters(), lr=lr)  # Corrected from opt to optimizer
                    loss = train_and_evaluate(model, train_loader, val_loader, optimizer, quantiles, epochs=100)  # Specified epochs for clarity
                    if loss < best_loss:
                        best_loss = loss
                        best_config = (batch_size, hidden_sizes, dropout_rate, optimizer_name, lr)
                        best_model = model
                        print(f"Best Configuration So far: {best_config} with Loss: {best_loss}")
                        print(f"Search done for {batch_size},{hidden_sizes},{dropout_rate},{optimizer_name},{lr}")
                    current_iteration += 1
                    completion_percentage = (current_iteration / total_iterations) * 100
                    print(f"Completed: {completion_percentage:.2f}%")

print(f"Best Configuration: {best_config} with Loss: {best_loss}")

Best Configuration So far: (128, (32, 64, 128), 0.0, 'Adam', 9.999999999999999e-06) with Loss: 6175512160.0
Search done for 128,(32, 64, 128),0.0,Adam,9.999999999999999e-06
Completed: 0.28%
Completed: 0.56%
Best Configuration So far: (128, (32, 64, 128), 0.0, 'Adam', 0.001) with Loss: 2681570368.0
Search done for 128,(32, 64, 128),0.0,Adam,0.001
Completed: 0.83%
Best Configuration So far: (128, (32, 64, 128), 0.0, 'Adam', 0.01) with Loss: 1069739308.0
Search done for 128,(32, 64, 128),0.0,Adam,0.01
Completed: 1.11%
Best Configuration So far: (128, (32, 64, 128), 0.0, 'Adam', 0.09999999999999999) with Loss: 1032999016.0
Search done for 128,(32, 64, 128),0.0,Adam,0.09999999999999999
Completed: 1.39%
Completed: 1.67%
Completed: 1.94%
Completed: 2.22%
Completed: 2.50%
Completed: 2.78%
Completed: 3.06%
Completed: 3.33%
Completed: 3.61%
Completed: 3.89%
Completed: 4.17%
Completed: 4.44%
Completed: 4.72%


In [ ]:
!pip install matplotlib
!pip install seaborn

import matplotlib.pyplot as plt
import seaborn as sns
import torch

def evaluate_and_plot_quantiles_direct(model, loader, quantiles):
    model.eval()
    all_preds = []
    all_targets = []

    # Collect predictions and targets
    with torch.no_grad():
        for data, target in loader:
            output = model(data)
            all_preds.append(output)
            all_targets.append(target)

    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    # Initialize counters for each quantile
    quantile_counts = [0] * (len(quantiles) + 1)  # Including one extra for above highest quantile

    # Assign each target to the highest quantile it does not exceed
    for i, target in enumerate(all_targets):
        # Find the highest quantile that the target value does not exceed
        for j in range(len(quantiles)):
            if target <= all_preds[i, j]:
                quantile_counts[j] += 1
                break
        else:
            # If none of the quantiles is greater than the target, it belongs to the last category
            quantile_counts[-1] += 1

    # Calculate percentages
    total_samples = len(all_targets)
    percentages = [count / total_samples * 100 for count in quantile_counts]

    # Plotting the results for visual confirmation
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(percentages)), percentages, color='skyblue')
    plt.xticks(ticks=range(len(percentages)), labels=[f'{q:.2f}' for q in quantiles] + ['>0.99'], rotation=45)
    plt.xlabel('Quantile Ranges')
    plt.ylabel('Percentage of Data Points')
    plt.title('Distribution of Data Points Across Quantiles')
    plt.show()

    return percentages

# Evaluate and visualize the distribution of data points across quantiles
percentages = evaluate_and_plot_quantiles_direct(best_model, val_loader, quantiles)

# Print the results
for i, pct in enumerate(percentages):
    if i < len(quantiles):
        print(f"Percentage of data <= {quantiles[i]:.2f} quantile: {pct:.2f}%")
    else:
        print(f"Percentage of data > {quantiles[-1]:.2f} quantile: {pct:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_selected_quantile_predictions_with_actuals(model, loader):
    model.eval()
    all_preds = []
    all_targets = []

    # Collect all predictions and actual targets
    with torch.no_grad():
        for data, target in loader:
            output = model(data)
            all_preds.append(output.numpy())  # Assuming the data fits in memory
            all_targets.append(target.numpy())

    # Convert lists to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)

    # Plotting
    plt.figure(figsize=(20, 10))
    quantiles_np = np.linspace(0.01, 0.99, num=all_preds.shape[1])

    # Select indices for lowest, middle, and highest quantile
    lowest_quantile_index = 0  # First quantile
    middle_quantile_index = all_preds.shape[1] // 2  # Middle quantile
    highest_quantile_index = all_preds.shape[1] - 1  # Last quantile

    # Create an x-axis for the data points
    x_axis = np.arange(all_preds.shape[0])

    # Plotting selected quantile predictions
    plt.plot(x_axis, all_preds[:, lowest_quantile_index], label=f'Lowest Quantile {quantiles_np[lowest_quantile_index]:.2f}', color='blue')
    plt.plot(x_axis, all_preds[:, middle_quantile_index], label=f'Middle Quantile {quantiles_np[middle_quantile_index]:.2f}', color='green')
    plt.plot(x_axis, all_preds[:, highest_quantile_index], label=f'Highest Quantile {quantiles_np[highest_quantile_index]:.2f}', color='purple')

    # Overlay actual data points as scatter plot
    plt.scatter(x_axis, all_targets, color='red', alpha=0.5, label='Actual Data', s=10)

    plt.title('Selected Quantile Predictions and Actual Data Points Indexed by Data Order')
    plt.xlabel('Index of Data Point')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.show()

# Assuming 'val_loader' is already defined and loaded with the appropriate data
plot_selected_quantile_predictions_with_actuals(best_model, val_loader)

In [ ]:
best_model.eval()
test_loss = 0
with torch.no_grad():
    for X_batch, y_batch in DataLoader(TensorDataset(X_test, y_test), batch_size=64):
        preds = best_model(X_batch)
        loss = quantile_loss(preds, y_batch, quantiles)
        test_loss += loss.item()

print(f'Test Loss: {test_loss / len(X_test)}')

In [ ]:
def evaluate_quantiles_individual(model, loader, quantiles):
    model.eval()
    all_preds = []
    all_targets = []

    # Collect predictions and actual targets
    with torch.no_grad():
        for data, target in loader:
            output = model(data)
            all_preds.append(output)
            all_targets.append(target)

    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    # Initialize quantile count
    quantile_counts = [0] * (len(quantiles) + 1)  # +1 for targets above the highest quantile

    # Assign each target to a quantile
    for target in all_targets:
        # Determine which quantile range each target belongs to
        # Compare target against all predictions and count how many quantiles it is greater than
        quantile_index = (target > all_preds).long().sum()  # How many quantiles are less than the target
        quantile_index = min(quantile_index, len(quantiles))  # Clamp to the number of quantiles to handle edge cases
        quantile_counts[quantile_index] += 1

    total_samples = float(all_targets.shape[0])
    percentages = [count / total_samples * 100 for count in quantile_counts]

    return percentages

# Evaluate and print the results
percentages = evaluate_quantiles_individual(best_model, val_loader, quantiles)

# Print results for each quantile
for i, pct in enumerate(percentages[:-1]):
    print(f"Percentage of data within the {quantiles[i] if i < len(quantiles) else 'last'} quantile: {pct:.2f}%")
if percentages:
    print(f"Percentage of data above the highest quantile {quantiles[-1]:.2f}: {percentages[-1]:.2f}%")


In [ ]:
import matplotlib.pyplot as plt

def evaluate_quantiles_with_overlap_check(model, loader, quantiles):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in loader:
            output = model(data)
            all_preds.append(output)
            all_targets.append(target)

    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    sorted_preds = all_preds.sort(dim=1).values

    # Check and print overlap
    overlaps = (sorted_preds[:, 1:] <= sorted_preds[:, :-1]).sum()
    if overlaps.item() > 0:
        print(f"Overlap detected in quantile predictions: {overlaps.item()} instances")

    quantile_counts = [0] * (sorted_preds.shape[1] - 1)
    for i in range(sorted_preds.shape[1] - 1):
        cond = (all_targets >= sorted_preds[:, i]) & (all_targets < sorted_preds[:, i + 1])
        quantile_counts[i] = cond.sum().item()

    total_samples = float(all_targets.shape[0])
    percentages = [count / total_samples * 100 for count in quantile_counts]
    total_percentage = sum(percentages)
    if abs(100.0 - total_percentage) > 0.1:
        print(f"Warning: Total percentage does not sum to 100% but is {total_percentage}%")

    return percentages

# Evaluate and check for overlaps
percentages = evaluate_quantiles_with_overlap_check(best_model, val_loader, quantiles)

# Print results
quantiles_np = quantiles.numpy()
for i in range(len(percentages)):
    print(f"Percentage of data between {quantiles_np[i]:.2f} and {quantiles_np[i+1]:.2f} quantiles: {percentages[i]:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch

def evaluate_and_plot_quantiles(model, loader, quantiles):
    model.eval()
    all_preds = []
    all_targets = []

    # Collect predictions and targets
    with torch.no_grad():
        for data, target in loader:
            output = model(data)  # Assuming this is outputting the quantiles directly
            all_preds.append(output)
            all_targets.append(target)

    # Convert collected data to single tensors
    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    # Sorting predictions along quantiles for each sample
    sorted_preds = all_preds.sort(dim=1).values

    # Plotting the quantile distributions
    plt.figure(figsize=(12, 8))
    quantiles_np = quantiles.numpy()  # Assuming quantiles is a torch tensor
    for i in range(sorted_preds.shape[1]):
        sns.kdeplot(sorted_preds[:, i].numpy(), label=f'Quantile {quantiles_np[i]:.2f}')
    plt.title('Distribution of Each Quantile Prediction')
    plt.xlabel('Predicted Values')
    plt.ylabel('Density')
    plt.legend()
    plt.show()

    # Check for overlaps
    overlaps = (sorted_preds[:, 1:] <= sorted_preds[:, :-1]).sum()
    if overlaps.item() > 0:
        print(f"Overlap detected in quantile predictions: {overlaps.item()} instances")

    # Calculating percentages
    quantile_counts = [0] * (sorted_preds.shape[1] - 1)
    for i in range(sorted_preds.shape[1] - 1):
        cond = (all_targets >= sorted_preds[:, i]) & (all_targets < sorted_preds[:, i + 1])
        quantile_counts[i] = cond.sum().item()

    total_samples = float(all_targets.shape[0])
    percentages = [count / total_samples * 100 for count in quantile_counts]
    total_percentage = sum(percentages)
    if abs(100.0 - total_percentage) > 0.1:
        print(f"Warning: Total percentage does not sum to 100% but is {total_percentage}%")

    return percentages

# Using the model, test loader and the defined function to evaluate and plot
percentages = evaluate_and_plot_quantiles(best_model, val_loader, quantiles)

# Printing percentages
for i in range(len(percentages)):
    print(f"Percentage of data between {quantiles[i]:.2f} and {quantiles[i+1]:.2f} quantiles: {percentages[i]:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import numpy as np

def plot_quantile_ranges(model, loader):
    model.eval()
    all_preds = []

    # Collect all predictions
    with torch.no_grad():
        for data, _ in loader:
            output = model(data)  # Assuming this is outputting the quantiles directly
            all_preds.append(output)

    # Convert collected data to a single tensor
    all_preds = torch.cat(all_preds, dim=0)

    # Sorting predictions along quantiles for each sample
    sorted_preds = all_preds.sort(dim=1).values

    # Create an x-axis for the data points
    x_axis = np.arange(sorted_preds.shape[0])

    # Plotting
    plt.figure(figsize=(20, 10))
    quantiles_np = np.linspace(0.01, 0.99, num=sorted_preds.shape[1])
    for i in range(sorted_preds.shape[1]):
        plt.plot(x_axis, sorted_preds[:, i].numpy(), label=f'Quantile {quantiles_np[i]:.2f}')

    plt.title('Quantile Predictions Across Data Index')
    plt.xlabel('Index of Data Point')
    plt.ylabel('Quantile Predicted Value')
    plt.legend(title='Quantiles', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Assuming 'val_loader' is already defined and loaded with the appropriate data
plot_quantile_ranges(best_model, val_loader)
